In [63]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
import os
import sys

sys.path.insert(0, os.getenv("HOME") + "/pycode/MscThesis/")
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    get_current_folders,
    get_folders_by_plate_id,
)

import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    sparse_to_doc,
)
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
)
from amftrack.pipeline.paths.directory import (
    run_parallel_transfer,
    find_state,
    directory_scratch,
    directory_project,
    directory_archive,
)
import dropbox
from amftrack.transfer.functions.transfer import upload, zip_file
from subprocess import call
from tqdm.autonotebook import tqdm

%env TEMP=/data/temp

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: TEMP=/data/temp


In [64]:
##### %autoreload 2

directory = "/run/user/357100554/gvfs/smb-share:server=sun.amolf.nl,share=shimizu-data,user=bisot/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE/"
directory = "/media/bisot/AMF_02/PRINCE/"
directory = "/mnt/sun/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE/"
directory = r"/mnt/sun-temp/TEMP/PRINCE_syncing/"
# directory = r'/run/user/357100554/gvfs/smb-share:server=prince.amolf.nl,share=d$/Data/Prince/Images/'

update_plate_info(directory)

all_folders = get_current_folders(directory)
# all_folders_drop = get_current_folders("dropbox")
all_folders_drop = get_current_folders("dropbox", True)
# folders = all_folders.loc[all_folders['strain']=="Agg"]
# folders = all_folders.loc[all_folders['Plate']==734]

analysed:   0%|          | 0/1241 [00:00<?, ?it/s]

In [55]:
all_folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,/Img/TileConfiguration.txt.registered,...,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,Temp,Temperature,InocDate
30051,/mnt/sun-temp/TEMP/PRINCE_syncing/20220315_015...,1007,1,'Carrot','C2','001P100N200C','Y','-',20220301,False,...,False,False,False,False,False,"15.03.2022, 01:52:",20220315_0152_Plate01,NaN,25,NaN
30052,/mnt/sun-temp/TEMP/PRINCE_syncing/20220314_225...,826,29,'Carrot','A5','001P100N100C','Y','-',20220304,False,...,False,False,False,False,False,"14.03.2022, 22:56:",20220314_2256_Plate29,NaN,25,NaN
30053,/mnt/sun-temp/TEMP/PRINCE_syncing/20220315_035...,875,37,'Carrot','A5','001P100N200C','Y','-',20220301,False,...,False,False,False,False,False,"15.03.2022, 03:57:",20220315_0357_Plate37,NaN,25,NaN
30054,/mnt/sun-temp/TEMP/PRINCE_syncing/20220315_024...,854,22,'Carrot','A5','001P100N200C','Y','-',20220222,False,...,False,False,False,False,False,"15.03.2022, 02:49:",20220315_0249_Plate22,NaN,25,NaN
30055,/mnt/sun-temp/TEMP/PRINCE_syncing/20220314_234...,R45,39,'Carrot','A5','001P100N100C','Y','-',20220308,False,...,False,False,False,False,False,"14.03.2022, 23:46:",20220314_2346_Plate39,NaN,25,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31154,/mnt/sun-temp/TEMP/PRINCE_syncing/20220322_124...,866,1,'Carrot','A5','001P100N200C','Y','-',20220315,False,...,False,False,False,False,False,"22.03.2022, 12:44:",20220322_1244_Plate01,NaN,25,NaN
31155,/mnt/sun-temp/TEMP/PRINCE_syncing/20220322_131...,983,10,'Carrot','C2','001P100N200C','Y','-',20220321,False,...,False,False,False,False,False,"22.03.2022, 13:19:",20220322_1319_Plate10,NaN,25,NaN
31156,/mnt/sun-temp/TEMP/PRINCE_syncing/20220322_070...,932,5,'Carrot','A5','001P100N200C','Y','-',20220315,False,...,False,False,False,False,False,"22.03.2022, 07:01:",20220322_0701_Plate05,NaN,25,NaN
31157,/mnt/sun-temp/TEMP/PRINCE_syncing/20220322_154...,907,9,'Carrot','A5','001P100N100C','Y','-',20220321,False,...,False,False,False,False,False,"22.03.2022, 15:46:",20220322_1546_Plate09,NaN,25,NaN


In [65]:
folders = all_folders.loc[all_folders["folder"] == "20220331_0111_Plate04"]

In [66]:
folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/skeleton_masked_compressed.mat,/Analysis/skeleton_pruned_compressed.mat,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder
2387,/mnt/sun-temp/TEMP/PRINCE_syncing/20220331_011...,887,4,'Carrot','A5','001P100N200C','Y','-',20220330,25,...,False,False,False,False,False,False,False,False,"31.03.2022, 01:11:",20220331_0111_Plate04


In [58]:
all_folders["strain"].unique()

array(["'C2'", "'A5'"], dtype=object)

In [68]:
folders = all_folders.loc[
    all_folders["folder"].isin(all_folders_drop["folder"]) == False
]
folders2 = all_folders.loc[
    all_folders["folder"].isin(all_folders_drop["folder"]) == True
]

In [20]:
from_surf = all_folders_drop.loc[
    all_folders_drop["total_path"].str.contains("projects")
]

In [21]:
folders = all_folders.loc[all_folders["folder"].isin(from_surf["folder"]) == False]

In [97]:
folders = all_folders.loc[
    all_folders["folder"].isin(all_folders_drop["folder"]) == False
]
folders = folders.loc[folders["Temp"].isin(["25", "30to25"])]
# folders = folders.loc[folders['Temp'].isin(['25'])]

In [ ]:
real_drop_folder = all_folders_drop.loc[all_folders_drop["size"] > 0.6]
folders = all_folders.loc[
    all_folders["folder"].isin(real_drop_folder["folder"]) == False
]

In [ ]:
folders

In [8]:
len(folders), len(all_folders_drop), len(all_folders)

(4139, 6164, 7649)

In [67]:
dir_drop = "prince_data"

In [70]:
run_info = folders.copy()
folder_list = list(run_info["folder"])
with tqdm(total=len(folder_list), desc="transferred") as pbar:
    for folder in folder_list:
        directory_name = folder
        run_info["unique_id"] = (
            run_info["Plate"].astype(str) + "_" + run_info["CrossDate"].astype(str)
        )
        line = run_info.loc[run_info["folder"] == directory_name]
        id_unique = line["unique_id"].iloc[0]

        path_snap = directory + directory_name
        API = str(np.load(os.getenv("HOME") + "/pycode/API_drop.npy"))

        path_zip = f'{os.getenv("TEMP")}/{directory_name}.zip'
        path_info = f'{os.getenv("TEMP")}/{directory_name}_info.json'
        line.to_json(path_info)
        zip_file(path_snap, path_zip)
        while True:
            try:
                upload(
                    API,
                    path_zip,
                    f"/{dir_drop}/{id_unique}/{directory_name}.zip",
                    chunk_size=256 * 1024 * 1024,
                )
                upload(
                    API,
                    path_info,
                    f"/{dir_drop}/{id_unique}/{directory_name}_info.json",
                    chunk_size=256 * 1024 * 1024,
                )
            except dropbox.exceptions.ApiError:
                continue
            break
        os.remove(path_info)
        os.remove(path_zip)
        pbar.update(1)

transferred:   0%|          | 0/1 [00:00<?, ?it/s]

zipping:   0%|          | 0/155 [00:00<?, ?it/s]

Uploaded:   0%|          | 0/845611707 [00:00<?, ?it/s]